In [6]:
import pandas as pd
import glob
import os
import keras

In [2]:
#save the models to model_folder
model_folder=''
#output folder of trajactory got from the simulation
output_folder=''
#create energy_folder to save the prediction results
energy_folder=''

# Select scenario for weather and vehicle class

In [3]:
#weather option: rainy, sunny, windy, and snowy
#Vclass option: diesel, hybrid, and electric
weather = 'rainy'
Vclass = 'diesel'

# Load microscopic energy eatimation model for the vehicle class


In [4]:
model =  keras.models.load_model(f'{model_folder}'+Vclass+'.model')

# predict energy consumption of each trips 

In [5]:
path = f'{output_folder}'                    
all_files = glob.glob(os.path.join(path, "Trajectory*.csv")) 
colnames=['time_ms','speed','acceleration','vehicle_ref','actorConfig_id','actorConfig_emissionClass','actorConfig_fuel','actorConfig_ref','actorConfig_vehicleClass']
for f in all_files:
    trip=pd.read_csv(f,names=colnames, header=None)
    trip = trip[trip.speed != 'speed']
    trip = trip[trip.acceleration != 'acceleration']
    trip['speed']=trip['speed'].astype(float)*0.036
    #km/h
    if Vclass == 'electric':
        trip['acceleration']=trip['acceleration'].astype(float)*0.001*3.28084
        #ft/s2
    else:
        trip['acceleration']=trip['acceleration'].astype(float)*0.001 
        #m/s2
    if weather == 'rainy':
        trip['temperature']= 60 
        trip['humidity']= 0.9 #range[0,1]
    if weather == 'sunny':
        trip['temperature']= 70
        trip['humidity']= 0.5
    if weather == 'windy':
        trip['temperature']= 60
        trip['humidity']= 0.6
    if weather == 'snowy':
        trip['temperature']= 30
        trip['humidity']= 0.4
    trip['roadGrade']= 0
    input2esti=trip[['speed','acceleration','temperature','humidity','roadGrade']]
    #prdiction 
    pre = model.predict(input2esti)
    tripf=pd.concat([trip,pd.DataFrame(pre,columns=['energy_'+Vclass])], axis=1) 
    #write the predicted results for each trip
    with open(f'{energy_folder}' + Vclass +'_'+ f[-44:-4] + '.csv', 'w', newline='') as oFile:
        tripf.to_csv(oFile, index = False)